# 01. Importing libraries

In [52]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02. Imorting data

In [53]:
# Create path
path = r'C:\Users\Quinn\OneDrive\Documents\CF - Data Analyst\Data Immersion\Instacart Basket Analysis'

In [54]:
# Import products data
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [55]:
# Import orders data
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = 0)

# 03. Checking dataconsistency for Products data

## Mixed typed data

In [56]:
# Checking mixed typed
for col in df_prods.columns.tolist():
  weird = (df_prods[[col]].map(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_prods[weird]) > 0:
    print (col)

product_name


### Observation: there are mixed data type values in the product_name columns because they are missing values, this can be handled in the next step

## Missing values

In [57]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

### There are 16 missing values in the product_name column

In [58]:
df_prods_nan = df_prods[df_prods['product_name'].isnull() == True]

In [59]:
df_prods_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


### These missing values happen in the product_name column, which cannnot be impute and there is no need to flag this information so these values will be dropped.

In [60]:
df_prods.shape

(49693, 5)

In [61]:
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [62]:
df_prods_clean.shape

(49677, 5)

## Duplicates

In [63]:
df_prods_dups = df_prods_clean[df_prods_clean.duplicated()]

In [64]:
df_prods_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


### There are 5 duplicates value in the cleaned product dataframe

In [65]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [66]:
df_prods_clean_no_dups.shape

(49672, 5)

# 04. Task: checking dataconsistency for Orders

## Q2. Run the df.describe() function on your df_ords dataframe.

In [67]:
df_ords.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### The count of the days_since_last_order columns is lower than other columns, indicating there are missing values 
### order_number column: mean is significantly higher than median, data distribution skewed to the right, indicating there are upper outliers in the data.
### days_since_last_order column: mean is greatly higher than median, data distribution also skewed to the right, indicating the presence of upper outliers.


## Mixed datatype

In [68]:
# Checking mixed typed
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

### There are no mixed type column in the orders dataframe

## Missing values

In [69]:
# Check for missing values in the df_ords
df_ords.isnull().sum()

order_id                      0
user_id                       0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

### There are 206209 missing values in the days_since_last_order. We cannot drop these values because it presents the first time customers order, instead, we should flag this information

In [70]:
# Create a flag variable for the NaN values in the days_since_last_order column
df_ords['first_order'] = np.where(np.isnan(df_ords['days_since_last_order'].values), 'First Order', 'Repeat Customer')

In [71]:
# Change datatype for the flag column
df_ords['first_order'] = df_ords['first_order'].astype('category')

In [72]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,first_order
0,2539329,1,1,2,8,NaN,First Order
1,2398795,1,2,3,7,15.0,Repeat Customer
2,473747,1,3,3,12,21.0,Repeat Customer
3,2254736,1,4,4,7,29.0,Repeat Customer
4,431534,1,5,4,15,28.0,Repeat Customer


## Duplicates

In [73]:
# Checking for duplicates values in the orders dataframe
df_ords_dups = df_ords[df_ords.duplicated()]

In [74]:
df_ords_dups

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,first_order


### There are no duplicated values

In [76]:
# Checking the final population of orders data
df_ords.shape

(3421083, 7)

# 04. Exporting Data

In [77]:
# Exporting orders data
df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.csv'))

In [78]:
# Exporting products data
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))